## RAG System Using Gemena With Hugging Face

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ['OPENAI_API_KEY '] = os.getenv('OPENAI_API_KEY')

In [2]:
### Install necessary package
! pip install pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 12.5 MB/s eta 0:00:00


In [3]:
### install langchain
!pip install -q transformers einops accelerate langchain bitsandbytes

In [4]:
### embedding with sentence transformer
!pip install install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.6 MB/s eta 0:00:00


In [2]:
!pip install llama_index

In [25]:
!pip install llama-index-llms-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 39.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.21.4
    Uninstalling huggingface-hub-0.21.4:
      Successfully uninstalled huggingface-hub-0.21.4


In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [2]:
document =  SimpleDirectoryReader('content/').load_data() ## read my document. It is an article about XAI
document

[Document(id_='a4b469f8-ab5e-4805-b751-ce7f7af37b7c', embedding=None, metadata={'page_label': '1', 'file_name': 'Business_data_challenge_report_group_4.pdf', 'file_path': '/home/onyxia/work/LLM_RAG/content/Business_data_challenge_report_group_4.pdf', 'file_type': 'application/pdf', 'file_size': 9992974, 'creation_date': '2024-03-10', 'last_modified_date': '2024-03-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='École nationale de la statistique\net de l’administration économique\nBusiness data challenge report - group 4\nPrediction of a property price\nStudents :\nGregoire HUBERT\nNicolas JULIEN\nNick jofrein TEDONZE\nSalah-Eddine EL MOUSLIH\nAxelNAOARINE\nApril 2023', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{conten

In [3]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [4]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="google/gemma-2b",
    model_name="google/gemma-2b",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

In [6]:
pip install llama-index-embeddings-langchain

Note: you may need to restart the kernel to use updated packages.


In [7]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

/tmp/ipykernel_47750/2250285859.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [8]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7efed8d686d0>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7efed8d686d0>, id_func=<function default_id_func at 0x7efee036d6c0>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7f0137c8c290>, callback_manager=<llama_index.core.callbacks.ba

In [9]:
index = VectorStoreIndex.from_documents(documents=document, service_context=service_context)

In [10]:
index

In [11]:
query_engine = index.as_query_engine()

In [22]:
response = query_engine.query('What is the goal of the business data challenge?')

In [23]:
print(response)


The goal of the business data challenge is to provide a tool for predicting the price of real
estate according to its characteristics (type of property, location, surface area,...). This tool
made available to customers of Meilleurtaux would allow them to estimate the real estate value
of their dwelling. Thus, this estimate is useful in two cases:
•In a mortgage loan project, when you need a bridging loan.
•In a credit consolidation project, when you have a mortgage guarantee.
---------------------
page_label: 43
file_path: /home/onyxia/work/LLM_RAG/content/Business_data_challenge_report_group_4.pdf

43

page_label: 3
file_path: /home/onyxia/work/LLM_RAG/content/Business_data_challenge_report_group_4.pdf

4
---------------------
Given the context information and not prior knowledge, answer the query.
Query: What is the goal of the business data challenge?
Answer: <|ASSISTANT|>
The goal of the business data challenge is to provide a tool for predicting the price of real
estate accordin